# Análise Exploratória de Dados

A análise exploratória de dados (em inglês: "Exploratory data analysis" - EDA) é uma das etapas mais importantes de um projeto de *Machine Learning*. Os objetivos desta etapa são:

- Conhecer o dataset em termos gerais
- Conhecer a natureza de cada feature
- Identificar anomalias e outliers
- Identificar features redundantes, features uteis, etc.

In [2]:
import pandas as pd

## Inicio: ler o dataset

Um dataset pequeno como o do exercício a seguir pode ser lido integralmente na memória RAM com um simples comando:

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/hsandmann/biblio/refs/heads/main/data/housing.csv')

Para datasets maiores existem outras bibliotecas que permitem trabalhar com arquivos imensos ou mesmo com datasets distribuidos em multiplos arquivos, como o [Dask Dataframes](https://docs.dask.org/en/stable/dataframe.html).

Com datasets tão grandes que nem cabem no disco e tem que ser armazenados em sistemas de arquivo distribuidos (como o [Amazon S3](https://aws.amazon.com/pt/pm/serv-s3/)) podemos usar o [Spark](https://spark.apache.org/) ou, novamente, o Dask Dataframes.

## Verificações iniciais

Vamos fazer verificações de "sanidade mental". O dataset que estamos utilizando é proveniente do capítulo 2 do livro texto da disciplina:

<img src="https://m.media-amazon.com/images/I/81qHV3ACapL._SL1500_.jpg" width=50%>


Este dataset contém informações agregadas sobre diferentes distritos residenciais do estado da Califórnia, nos Estados Unidos, que foram coletadas nos anos 1990. Cada linha do dataset representa um distrito residencial, e as colunas contêm diversas características socioeconômicas e geográficas desses distritos, bem como o valor médio das casas nesses locais.

Trata-se de um dataset famoso na comunidade de Ciência dos Dados, comumente chamado de ["California Housing"](https://lib.stat.cmu.edu/datasets/). A versão aqui incluida contém modificações em relação ao dataset original, para que possamos exercitar uma ampla variedade de cenários.

Para realizar os exercícios a seguir consulte a documentação da biblioteca [Pandas](https://pandas.pydata.org/)

In [4]:
df

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,INLAND


As dimensões do dataset podem ser obtidos por:

In [5]:
df.shape

(20640, 10)

In [6]:
df.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'ocean_proximity'],
      dtype='object')

**Exercício**

- Quantos *exemplos* existem na base de dados?
- Quantas colunas?
- O que cada coluna significa?
- Qual o tipo de cada coluna: contínua ou categórica?

No dataset deste exercício, a coluna `median_house_value` será usada como o *target*: o valor que desejamos prever. As demais colunas serão as *features*. Portanto, nosso problema é de **regressão**: queremos prever valores contínuos.

Um pouco de nomenclatura:

- Neste curso, seguindo a nomenclatura do livro texto, vamos denotar o número de exemplos no dataset por $m$, e o número de *features* por $n$.
- Desta forma, nossa tabela de *features* será uma *matriz* de tamanho $m \times n$. É costumeiro usar a letra $X$ para indicar essa matriz de $m$ exemplos por $n$ *features*.
- O target é um vetor com o mesmo número de posições que o número de exemplos, ou seja, um vetor de tamanho $m$. Usamos a letra $y$ para indicar o vetor *target*.

Vamos separar as *features* do *target* neste dataset:

In [7]:
X = df.drop('median_house_value', axis=1).copy()
y = df['median_house_value'].copy() #target

X

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,NEAR BAY
...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,INLAND


## Separação treino-teste

Conforme vimos na aula anterior, é importante ter dois conjunto de exemplos independentes, e provenientes da mesma *população*:

- O **conjunto de treino**: permite ensinar o computador a partir de exemplos, ou seja, *treinar* o modelo.
- O **conjunto de teste**: usado para medir o desempenho do modelo treinado em um dataset independente.

**Exercício**

Relembre as discussões da aula passada e enuncie os seguintes conceitos:

- Underfitting
- Overfitting

A biblioteca [Scikit-Learn](https://scikit-learn.org/) contém vários componentes para ajudar em nossos projetos de Machine Learning: modelos, processadores de dados, ferramentas de avaliação e seleção de parâmetros, implementação de modelos, etc. Será a principal ferramenta do nosso curso. Vamos usar o Scikit-Learn para dividir o dataset original em duas partes: um conjunto de treino e um de teste.

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
)

In [ ]:
X.shape, y.shape

((20640, 9), (20640,))

In [ ]:
X_train.shape, y_train.shape

((16512, 9), (16512,))

In [ ]:
X_test.shape, y_test.shape

((4128, 9), (4128,))

**Exercício**

Pesquise sobre "geração de números pseudo-aleatórios" e explique o papel do argumento `random_state` na função acima, e sua importância.

Mas por que fizemos a separação treino-teste antes de qualquer análise, se nem ao menos vamos construir um modelo preditivo nesta aula?

Novamente, lembre-se da aula anterior. É importante que **o conjunto de teste não deve influenciar o processo de treino**, para que possamos conhecer o desempenho real do nosso sistema em amostras *novas*, não usadas no treino.

O que aconteceria se fizéssemos uma análise exploratória no conjunto completo (treino junto com teste) de dados? Estaríamos "dando uma espiadinha" (*snooping*) no conjunto de teste, e isso poderia impactar nossas *escolhas* na construção do modelo preditivo e no processo de teste, contrariando o princípio acima! Este erro processual chama-se *data snooping*, e deve ser evitado.

Logo, vamos adotar os seguintes princípios:

- **O conjunto de treino pode ser usado à vontade**;
- **O conjunto de teste É SAGRADO e vai ser usado apenas para um propósito: medir o desempenho preditivo do modelo treinado**. Não olhe para ele, não chegue perto, não tome conhecimento da existência dele antes da hora!

## Análise univariada

Vamos usar as ferramentas da análise exploratória para conhecer melhor nossos dados.

- **Medidas descritivas para variáveis contínuas**
    - **Posição**: média, mediana, moda
    - **Espalhamento**: desvio padrão, intervalo inter-quartil, valores máximo e mínimo

- **Medidas descritivas para variáveis categóricas**
    - **Frequência**: contagem de número de exemplos por categoria

- **Visualização**
    - **variáveis contínuas**: histograma, boxplot
    - **variáveis categóricas**: gráfico de barras

**Exercício**

Gere as métricas descritivas e as visualizações para cada *feature* e para o *target* no conjunto de treino.

O que estamos procurando?

- Anomalias: dados faltantes, fenômenos de saturação
- Outliers:
    - Variáveis contínuas: valores muito fora do normal
    - Variáveis categóricas: categorias com frequência muito baixa ou muito alta, com grande disparidade em relação às demais
- Erros grosseiros, problemas com conversão de dados
    - Acontece com muita frequência, especialmente para dados que não tem padronização universalmente aceita como datas

Além destes comportamentos anômalos, devemos também observar o comportamento normal das variáveis:

- A distribuição (histograma) é simétrica ou assimétrica? Possui cauda longa para algum dos lados?
- A variável apresenta valores positivos e negativos, ou apenas valores positivos (ou apenas negativos)? É bastante concentrada ou espalhada?

**Exercício**

Analise as *features* e o *target* deste dataset.

## Análise bivariada

Primeiro vamos fazer uma análise descritiva. As principais ferramentas aqui são:

- Entre variáveis contínuas: [**correlação**](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient);
- Entre variáveis categóricas: [**coeficiente V de Cramér**](https://en.wikipedia.org/wiki/Cram%C3%A9r%27s_V);
- Entre uma variável contínua e uma categórica: [**teste de Kruskal-Wallis**](https://en.wikipedia.org/wiki/Kruskal%E2%80%93Wallis_test).

**Exercício**

Faça a análise descritiva e determine pares de variáveis com alta associação.

Agora vamos fazer visualizações para:

- pares de *features* com alta associação
- *target* versus cada uma das *features*

Use as ferramentas adequadas:

- Entre variáveis contínuas: **gráfico de espalhamento**;
- Entre variáveis categóricas: **matriz de contingência**;
- Entre uma variável contínua e uma categórica: **boxplot da contínua por categoria da categórica**;

**Exercício**

Você sabe o que fazer

Assim como na análise univariada, procure anomalias, outliers, etc.. Observe também:

- *Features* com associação muito **forte** entre si: pode indicar redundância de *features*. Em casos extremos, podemos descobrir que duas *features* são, na verdade, a mesma *feature*, e portanto uma delas deve ser eliminada. Exemplos:
    - Temperatura em $C$, $F$ e $K$
    - Preço em R$ e US$ (supondo estabilidade da cotação, é claro)
    - As vezes a similaridade não é tão óbvia se não prestamos atenção. Por exemplo: peso e volume (coisas bem diferentes) em uma situação em que a densidade é a mesma (e.g. peso de um caminhão-tanque e volume de combustível, se os caminhões são todos do mesmo modelo).
- *Features* com associação muito **fraca** entre si: isso é ótimo, são provavelmente informações complementares!
- *Features* com associação muito **forte** com o *target*: isso é bom, indica uma *feature* que pode ser que tenha um bom valor preditivo. Mas... ***CUIDADO***!!! É raro mas acontece muito (kkkkk!) casos em que a *feature*... é o *target* disfarçado! Em geral, se a capacidade preditiva do seu modelo é:
    - muito ruim: pode ser assim mesmo ou pode ser que você fez algum erro
    - mais ou menos: é mais um dia de trabalho do cientista de dados, normal
    - boa: deu sorte, hein?
    - muito boa: esmola demais, o santo desconfia... se o seu modelo de repente é perfeito, certamente tem alguma *feature* que é o *target* de chapeu e bigodinho, acredite!
- *Features* com associação muito **fraca** com o *target*: podem ser *features* lixo, a serem descartadas. Nada em Machine Learning é muito definitivo, em geral é melhor testar tudo, mas se for necessário eliminar *a priori* alguma *feature* (pois excesso de features com falta de volume de exemplos impede o bom treinamento), esse pode ser um critério.

**Exercício**

Determine as melhores *features* em termos de sua relação com o *target*